In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

### 데이터 로드 (q: 15분 단위, h: 1시간 단위)

In [2]:
generation_q = pd.read_csv("C:/Users/symply_jina/Desktop/seohyun/UIC4801/DATA_generation.csv")
generation_q['Time'] = pd.to_datetime(generation_q['Time'], format='%Y-%m-%d %H:%M')
generation_q['Hour'] = generation_q['Time'].dt.floor('H') 
generation_h = generation_q.groupby('Hour').sum(numeric_only=True)  

demand_q = pd.read_csv("C:/Users/symply_jina/Desktop/seohyun/UIC4801/DATA_demand.csv")
demand_q['Time'] = pd.to_datetime(demand_q['Time'], format='%Y-%m-%d %H:%M')
demand_q['Hour'] = demand_q['Time'].dt.floor('H')   
demand_h = demand_q.groupby('Hour').sum(numeric_only=True)  

price_q = pd.read_csv("C:/Users/symply_jina/Desktop/seohyun/UIC4801/DATA_price.csv")
price_q['Time'] = pd.to_datetime(price_q['Time'], format='%Y-%m-%d %H:%M')

### Set

In [3]:
I = list(range(len(generation_q.columns) - 2))
T = list(generation_q['Time'].dt.hour.unique())
S = list(range(20))
prob = {s: 1 / len(S) for s in S}

### 데이터 전처리

#### Generation

In [4]:
generation_avg = {
    (i, t): generation_h[generation_h.index.hour == t].mean()[i] for i in I for t in T
}

def generate_randomized_generation(I, T, S, generation_avg, randomness_level):
    np.random.seed(7)
    if randomness_level == 'low':
        noise_factors = {(i, t, s): np.random.uniform(0.8, 1.2) for i in I for t in T for s in S}
    elif randomness_level == 'medium':
        noise_factors = {(i, t, s): np.random.uniform(0.5, 1.5) for i in I for t in T for s in S}
    elif randomness_level == 'high':
        noise_factors = {(i, t, s): np.random.uniform(0.2, 1.8) for i in I for t in T for s in S}
    else:
        raise ValueError("Invalid randomness level. Please choose 'low', 'medium', or 'high'.")
    
    generation_r = {
        (i, t, s): generation_avg[(i, t)]*noise_factors[(i, t, s)] for i in I for t in T for s in S
    }
    return generation_r

#### Demand (randomized)

In [5]:
demand_avg = {
    (i, t): demand_h[demand_h.index.hour == t].mean()[i] for i in I for t in T
}

def generate_randomized_demand(I, T, S, demand_avg, randomness_level):
    np.random.seed(17)
    if randomness_level == 'low':
        noise_factors = {(i, t, s): np.random.uniform(0.8, 1.2) for i in I for t in T for s in S}
    elif randomness_level == 'medium':
        noise_factors = {(i, t, s): np.random.uniform(0.5, 1.5) for i in I for t in T for s in S}
    elif randomness_level == 'high':
        noise_factors = {(i, t, s): np.random.uniform(0.2, 1.8) for i in I for t in T for s in S}
    else:
        raise ValueError("Invalid randomness level. Please choose 'low', 'medium', or 'high'.")
    
    demand_r = {
        (i, t, s): demand_avg[(i, t)]*noise_factors[(i, t, s)] for i in I for t in T for s in S
    }
    return demand_r

#### Randomize

In [6]:
generation_r = generate_randomized_generation(I, T, S, generation_avg, 'medium')
demand_r = generate_randomized_demand(I, T, S, demand_avg, 'medium')

#### Residual = generation - demand

In [7]:
residual = {(i, t, s): generation_r[(i, t, s)] - demand_r[(i, t, s)] for i in I for t in T for s in S}

R = {
    (i, t, s): max(0, residual[(i, t, s)])
    for i in I for t in T for s in S
}

### Price 설정

In [8]:
unique_days = generation_q['Time'].dt.normalize().unique()
days = len(unique_days)
D = list(range(min(len(S), days)))

price_q['Hour'] = price_q['Time'].dt.floor('H')    # 시간으로 그룹화
price_h = price_q.groupby('Hour').mean(numeric_only=True)  # 각 시간대별로 평균 계산

price = price_h.iloc[:len(D)*len(T)]

P_DA = {t: sum(price['Price'].iloc[t + d * len(T)] for d in D) / len(D) * 1.2 for t in T}
P_RT = {(t, d): price['Price'].iloc[t + d * len(T)] for t in T for d in D}
P_PN = {t: sum(price['Price'].iloc[t + d * len(T)] for d in D) / len(D) * 2 for t in T}

### 모델 DER with Aggregation (Part: Aggregation)

In [9]:
alpha_DA = {t: cp.Variable(pos=True) for t in T}
beta_plus = {(t, s): cp.Variable(pos=True) for t in T for s in S}
beta_minus = {(t, s): cp.Variable(pos=True) for t in T for s in S}

In [10]:
obj_case2 = sum(
    P_DA[t] * alpha_DA[t] for t in T
) + sum(
    prob[s] * (P_RT[t, s] * beta_plus[t, s] - P_PN[t] * beta_minus[t, s])
    for t in T for s in S
)

constraints_case2 = []

for t in T:
    for s in S:
        constraints_case2.append(sum(R[i, t, s] for i in I) - alpha_DA[t] == beta_plus[t, s] - beta_minus[t, s])

for t in T:
    for s in S:
        constraints_case2.append(sum(R[i,t,s] for i in I) >= beta_plus[t, s])

M = max(sum(R[i,t,s] for i in I) for t in T for s in S)
z_case2 = {(t, s): cp.Variable(boolean=True) for t in T for s in S}
for t in T:
    for s in S:
            constraints_case2.append(beta_plus[t, s] <= M * z_case2[t, s])
            constraints_case2.append(beta_minus[t, s] <= M * (1 - z_case2[t, s]))

prob_case2 = cp.Problem(cp.Maximize(obj_case2), constraints_case2)
prob_case2.solve(solver='GUROBI', TimeLimit=60*30, IntFeasTol=1e-9)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-25


16153.516356625341

In [11]:
print("Case 2 최적해:")
print(f"총 이익: {prob_case2.value}")

Case 2 최적해:
총 이익: 16153.516356625341


### 결과 분석

#### 수익 분석

In [12]:
# Day-ahead 수익 비교
total_da_profit_obj = 0
for t in T:
    total_da_profit_obj += P_DA[t] * alpha_DA[t].value

# Real-time 수익 비교
total_rt_profit_obj = 0
for t in T:
    for s in S:
        rt_profit_obj = P_RT[t, s] * beta_plus[t, s].value
        total_rt_profit_obj += prob[s] * rt_profit_obj

# 패널티 비용 비교
total_penalty_cost_obj = 0
for t in T:
    for s in S:
        penalty_cost_obj = P_PN[t] * beta_minus[t, s].value
        total_penalty_cost_obj += prob[s] * penalty_cost_obj

# 총 시스템 이익 (목적 함수 기반)
total_system_profit_obj = total_da_profit_obj + total_rt_profit_obj - total_penalty_cost_obj

# 결과 출력
print(f"총 Day-ahead 수익 (_obj): {total_da_profit_obj:.2f}")
print(f"총 Real-time 수익 (_obj): {total_rt_profit_obj:.2f}")
print(f"총 Penalty 비용 (_obj): {total_penalty_cost_obj:.2f}")
print(f"목적 함수 기반 총 이익 (_obj): {total_system_profit_obj:.2f}")

총 Day-ahead 수익 (_obj): 13535.52
총 Real-time 수익 (_obj): 3032.59
총 Penalty 비용 (_obj): 414.60
목적 함수 기반 총 이익 (_obj): 16153.52


#### 하루 aggregated 커밋량 분석

In [13]:
# 각 t에 대한 alpha, beta +,-의 시나리오 평균 계산
for t in T:
    avg_alpha = alpha_DA[t].value
    avg_beta_plus = sum(beta_plus[t, s].value for s in S) / len(S)
    avg_beta_minus = sum(beta_minus[t, s].value for s in S) / len(S)
    print(f"시간 {t}: alpha: {avg_alpha:.2f}, beta+: {avg_beta_plus:.2f}, beta-: {avg_beta_minus:.2f}")

print(sum(alpha_DA[t].value for t in T))

시간 0: alpha: 0.00, beta+: 0.00, beta-: 0.00
시간 1: alpha: 0.00, beta+: 0.00, beta-: 0.00
시간 2: alpha: 0.00, beta+: 0.00, beta-: 0.00
시간 3: alpha: 0.00, beta+: 0.00, beta-: 0.00
시간 4: alpha: 0.00, beta+: 0.00, beta-: 0.00
시간 5: alpha: 0.00, beta+: 0.00, beta-: 0.00
시간 6: alpha: 0.00, beta+: 0.00, beta-: 0.00
시간 7: alpha: 0.00, beta+: 0.00, beta-: 0.00
시간 8: alpha: 0.00, beta+: 0.61, beta-: 0.00
시간 9: alpha: 9.52, beta+: 2.68, beta-: 0.41
시간 10: alpha: 32.29, beta+: 7.23, beta-: 0.73
시간 11: alpha: 57.22, beta+: 11.05, beta-: 0.44
시간 12: alpha: 68.46, beta+: 11.98, beta-: 0.95
시간 13: alpha: 64.78, beta+: 17.59, beta-: 0.43
시간 14: alpha: 53.12, beta+: 13.11, beta-: 0.66
시간 15: alpha: 38.29, beta+: 7.82, beta-: 1.90
시간 16: alpha: 9.89, beta+: 8.88, beta-: 0.49
시간 17: alpha: 0.69, beta+: 1.56, beta-: 0.09
시간 18: alpha: 0.00, beta+: 0.02, beta-: 0.00
시간 19: alpha: 0.00, beta+: 0.00, beta-: 0.00
시간 20: alpha: 0.00, beta+: 0.00, beta-: 0.00
시간 21: alpha: 0.00, beta+: 0.00, beta-: 0.00
시간 22: alp

### 결과 저장

In [14]:
alpha_DA_df = pd.DataFrame({'alpha_DA': [alpha_DA[t].value for t in T]})
alpha_DA_df.to_csv('result_alpha_DA.csv', index=False)

agg_value = pd.DataFrame({'agg_value': [prob_case2.value]})
agg_value.to_csv('result_agg_value.csv', index=False)